In [1]:
library(gtsummary)
library(dplyr)
library(stringr)
library(lubridate)
library(flextable)


Attaching package: ‘dplyr’


The following objects are masked from ‘package:stats’:

    filter, lag


The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union



Attaching package: ‘lubridate’


The following objects are masked from ‘package:base’:

    date, intersect, setdiff, union


Warning message:
“package ‘flextable’ was built under R version 4.3.3”

Attaching package: ‘flextable’


The following object is masked from ‘package:gtsummary’:

    continuous_summary




In [2]:
# 1. Read Data
raw_data <- read.csv("data/data.csv")

In [3]:
# 2. Clean and Define Variables
table1_data <- raw_data %>%
  mutate(
    # --- AGE ---
    Date_Diagnosis = ymd(Fecha_diag),
    Age = as.numeric(Anio_diag) - as.numeric(Anio_nac),

    # --- SEX ---
    Sex = factor(Sexo, levels = c("Mujer", "Hombre"), labels = c("Female", "Male")),

    # --- RESIDENCE ---
    Residence = if_else(Parr_res == "QUITO", "Urban (Quito)", "Rural (Parishes)"),

    # --- EDUCATION ---
    Education = case_when(
      Nivel_educ == "Ninguna" ~ "None / Illiterate",
      Nivel_educ == "Primaria" ~ "Primary",
      Nivel_educ == "Secundaria" ~ "Secondary",
      Nivel_educ == "Superior" ~ "Higher (University/Technical)",
      TRUE ~ "Missing/Unknown"
    ),
    Education = factor(Education, levels = c("None / Illiterate", "Primary", "Secondary", "Higher (University/Technical)", "Missing/Unknown")),

    # --- ESTABLISHMENT ---
    Establishment = case_when(
      Establecimiento == "IESS" ~ "Social Security (IESS)",
      Establecimiento == "MSP" ~ "Ministry of Public Health (MSP)",
      Establecimiento == "SOLCA" ~ "Specialized Cancer Center (SOLCA)",
      Establecimiento == "Privados no SOLCA" ~ "Private Sector",
      TRUE ~ "Missing"
    ),

    # --- HISTOLOGY (CRITICAL FIX) ---
    # We convert to Upper Case to match safely
    Histology_Raw = toupper(Morf_tumor_CIEO3),

    Subtype = case_when(
      # 1. Hodgkin: Must contain "HODGKIN" but NOT "NO HODGKIN"
      str_detect(Histology_Raw, "HODGKIN") & !str_detect(Histology_Raw, "NO HODGKIN") ~ "Hodgkin Lymphoma",

      # 2. DLBCL: Contains "DIFUSO" (Diffuse)
      str_detect(Histology_Raw, "DIFUSO") ~ "Diffuse Large B-Cell (DLBCL)",

      # 3. Follicular
      str_detect(Histology_Raw, "FOLICULAR") ~ "Follicular Lymphoma",

      # 4. Burkitt
      str_detect(Histology_Raw, "BURKITT") ~ "Burkitt Lymphoma",

      # 5. T-Cell / NK-Cell
      str_detect(Histology_Raw, "T") | str_detect(Histology_Raw, "NK") ~ "Mature T/NK-Cell",

      # 6. Others/NOS (Everything else)
      TRUE ~ "B-Cell / NOS / Other"
    ),

    # --- PRIOR TUMORS ---
    Prior_Tumors = case_when(
      Num_tumores == 0 ~ "No prior tumor",
      Num_tumores == 1 ~ "One prior tumor",
      Num_tumores >= 2 ~ "Two or more prior tumors"
    ),
      
    # --- PRIMARY TUMOR SITE (Precise Mapping) ---
    Tumor_Site = case_when(
      grepl("^C77", Localiz_tumor_r) ~ "Nodal",
      grepl("^C16", Localiz_tumor_r) ~ "Gastric (Extranodal)",
      TRUE ~ "Other Extranodal"
    )
  ) %>%
  filter(Age >= 18)

In [4]:
# 3. Generate Table
# Update the tbl_summary call to include the "%" symbol
t1_corrected <- table1_data %>%
  select(Sex, Age, Residence, Education, Establishment, Subtype, Prior_Tumors, Tumor_Site) %>%
  tbl_summary(
    statistic = list(all_categorical() ~ "{n} ({p}%)"), # Added % here
    digits = all_categorical() ~ 1,
    label = list(
      Sex ~ "Sex",
      Age ~ "Age at Diagnosis",
      Residence ~ "Residence Area",
      Education ~ "Education Level",
      Establishment ~ "Diagnostic Establishment",
      Subtype ~ "Histological Subtype",
      Prior_Tumors ~ "History of Prior Cancer",
      Tumor_Site ~ "Primary Tumor Site"
    ),
    missing_text = "Missing"
  ) %>%
  bold_labels()

#t1_corrected

In [5]:
# 4. Export to Microsoft Word (Professional Manuscript Format)
t1_corrected %>%
  as_flex_table() %>%
  # Professional styling for Word
  fontsize(size = 9, part = "all") %>%
  autofit() %>%
  save_as_docx(path = "tables/Table1_Sociodemographics.docx")

In [6]:
# 5. Export to HTML (Best for keeping the JAMA look)
t1_corrected %>%
  as_gt() %>%
  gt::gtsave("tables/Table1_Sociodemographics.html")

print("Files saved successfully as 'tables/Table1_Sociodemographics.docx' and 'Table1_Sociodemographics.html'")

[1] "Files saved successfully as 'tables/Table1_Sociodemographics.docx' and 'Table1_Sociodemographics.html'"


<div id="ybtyhdwwsl" style="padding-left:0px;padding-right:0px;padding-top:10px;padding-bottom:10px;overflow-x:auto;overflow-y:auto;width:auto;height:auto;">
  <style>#ybtyhdwwsl table {
  font-family: system-ui, 'Segoe UI', Roboto, Helvetica, Arial, sans-serif, 'Apple Color Emoji', 'Segoe UI Emoji', 'Segoe UI Symbol', 'Noto Color Emoji';
  -webkit-font-smoothing: antialiased;
  -moz-osx-font-smoothing: grayscale;
}

#ybtyhdwwsl thead, #ybtyhdwwsl tbody, #ybtyhdwwsl tfoot, #ybtyhdwwsl tr, #ybtyhdwwsl td, #ybtyhdwwsl th {
  border-style: none;
}

#ybtyhdwwsl p {
  margin: 0;
  padding: 0;
}

#ybtyhdwwsl .gt_table {
  display: table;
  border-collapse: collapse;
  line-height: normal;
  margin-left: auto;
  margin-right: auto;
  color: #333333;
  font-size: 16px;
  font-weight: normal;
  font-style: normal;
  background-color: #FFFFFF;
  width: auto;
  border-top-style: solid;
  border-top-width: 2px;
  border-top-color: #A8A8A8;
  border-right-style: none;
  border-right-width: 2px;
  border-right-color: #D3D3D3;
  border-bottom-style: solid;
  border-bottom-width: 2px;
  border-bottom-color: #A8A8A8;
  border-left-style: none;
  border-left-width: 2px;
  border-left-color: #D3D3D3;
}

#ybtyhdwwsl .gt_caption {
  padding-top: 4px;
  padding-bottom: 4px;
}

#ybtyhdwwsl .gt_title {
  color: #333333;
  font-size: 125%;
  font-weight: initial;
  padding-top: 4px;
  padding-bottom: 4px;
  padding-left: 5px;
  padding-right: 5px;
  border-bottom-color: #FFFFFF;
  border-bottom-width: 0;
}

#ybtyhdwwsl .gt_subtitle {
  color: #333333;
  font-size: 85%;
  font-weight: initial;
  padding-top: 3px;
  padding-bottom: 5px;
  padding-left: 5px;
  padding-right: 5px;
  border-top-color: #FFFFFF;
  border-top-width: 0;
}

#ybtyhdwwsl .gt_heading {
  background-color: #FFFFFF;
  text-align: center;
  border-bottom-color: #FFFFFF;
  border-left-style: none;
  border-left-width: 1px;
  border-left-color: #D3D3D3;
  border-right-style: none;
  border-right-width: 1px;
  border-right-color: #D3D3D3;
}

#ybtyhdwwsl .gt_bottom_border {
  border-bottom-style: solid;
  border-bottom-width: 2px;
  border-bottom-color: #D3D3D3;
}

#ybtyhdwwsl .gt_col_headings {
  border-top-style: solid;
  border-top-width: 2px;
  border-top-color: #D3D3D3;
  border-bottom-style: solid;
  border-bottom-width: 2px;
  border-bottom-color: #D3D3D3;
  border-left-style: none;
  border-left-width: 1px;
  border-left-color: #D3D3D3;
  border-right-style: none;
  border-right-width: 1px;
  border-right-color: #D3D3D3;
}

#ybtyhdwwsl .gt_col_heading {
  color: #333333;
  background-color: #FFFFFF;
  font-size: 100%;
  font-weight: normal;
  text-transform: inherit;
  border-left-style: none;
  border-left-width: 1px;
  border-left-color: #D3D3D3;
  border-right-style: none;
  border-right-width: 1px;
  border-right-color: #D3D3D3;
  vertical-align: bottom;
  padding-top: 5px;
  padding-bottom: 6px;
  padding-left: 5px;
  padding-right: 5px;
  overflow-x: hidden;
}

#ybtyhdwwsl .gt_column_spanner_outer {
  color: #333333;
  background-color: #FFFFFF;
  font-size: 100%;
  font-weight: normal;
  text-transform: inherit;
  padding-top: 0;
  padding-bottom: 0;
  padding-left: 4px;
  padding-right: 4px;
}

#ybtyhdwwsl .gt_column_spanner_outer:first-child {
  padding-left: 0;
}

#ybtyhdwwsl .gt_column_spanner_outer:last-child {
  padding-right: 0;
}

#ybtyhdwwsl .gt_column_spanner {
  border-bottom-style: solid;
  border-bottom-width: 2px;
  border-bottom-color: #D3D3D3;
  vertical-align: bottom;
  padding-top: 5px;
  padding-bottom: 5px;
  overflow-x: hidden;
  display: inline-block;
  width: 100%;
}

#ybtyhdwwsl .gt_spanner_row {
  border-bottom-style: hidden;
}

#ybtyhdwwsl .gt_group_heading {
  padding-top: 8px;
  padding-bottom: 8px;
  padding-left: 5px;
  padding-right: 5px;
  color: #333333;
  background-color: #FFFFFF;
  font-size: 100%;
  font-weight: initial;
  text-transform: inherit;
  border-top-style: solid;
  border-top-width: 2px;
  border-top-color: #D3D3D3;
  border-bottom-style: solid;
  border-bottom-width: 2px;
  border-bottom-color: #D3D3D3;
  border-left-style: none;
  border-left-width: 1px;
  border-left-color: #D3D3D3;
  border-right-style: none;
  border-right-width: 1px;
  border-right-color: #D3D3D3;
  vertical-align: middle;
  text-align: left;
}

#ybtyhdwwsl .gt_empty_group_heading {
  padding: 0.5px;
  color: #333333;
  background-color: #FFFFFF;
  font-size: 100%;
  font-weight: initial;
  border-top-style: solid;
  border-top-width: 2px;
  border-top-color: #D3D3D3;
  border-bottom-style: solid;
  border-bottom-width: 2px;
  border-bottom-color: #D3D3D3;
  vertical-align: middle;
}

#ybtyhdwwsl .gt_from_md > :first-child {
  margin-top: 0;
}

#ybtyhdwwsl .gt_from_md > :last-child {
  margin-bottom: 0;
}

#ybtyhdwwsl .gt_row {
  padding-top: 8px;
  padding-bottom: 8px;
  padding-left: 5px;
  padding-right: 5px;
  margin: 10px;
  border-top-style: solid;
  border-top-width: 1px;
  border-top-color: #D3D3D3;
  border-left-style: none;
  border-left-width: 1px;
  border-left-color: #D3D3D3;
  border-right-style: none;
  border-right-width: 1px;
  border-right-color: #D3D3D3;
  vertical-align: middle;
  overflow-x: hidden;
}

#ybtyhdwwsl .gt_stub {
  color: #333333;
  background-color: #FFFFFF;
  font-size: 100%;
  font-weight: initial;
  text-transform: inherit;
  border-right-style: solid;
  border-right-width: 2px;
  border-right-color: #D3D3D3;
  padding-left: 5px;
  padding-right: 5px;
}

#ybtyhdwwsl .gt_stub_row_group {
  color: #333333;
  background-color: #FFFFFF;
  font-size: 100%;
  font-weight: initial;
  text-transform: inherit;
  border-right-style: solid;
  border-right-width: 2px;
  border-right-color: #D3D3D3;
  padding-left: 5px;
  padding-right: 5px;
  vertical-align: top;
}

#ybtyhdwwsl .gt_row_group_first td {
  border-top-width: 2px;
}

#ybtyhdwwsl .gt_row_group_first th {
  border-top-width: 2px;
}

#ybtyhdwwsl .gt_summary_row {
  color: #333333;
  background-color: #FFFFFF;
  text-transform: inherit;
  padding-top: 8px;
  padding-bottom: 8px;
  padding-left: 5px;
  padding-right: 5px;
}

#ybtyhdwwsl .gt_first_summary_row {
  border-top-style: solid;
  border-top-color: #D3D3D3;
}

#ybtyhdwwsl .gt_first_summary_row.thick {
  border-top-width: 2px;
}

#ybtyhdwwsl .gt_last_summary_row {
  padding-top: 8px;
  padding-bottom: 8px;
  padding-left: 5px;
  padding-right: 5px;
  border-bottom-style: solid;
  border-bottom-width: 2px;
  border-bottom-color: #D3D3D3;
}

#ybtyhdwwsl .gt_grand_summary_row {
  color: #333333;
  background-color: #FFFFFF;
  text-transform: inherit;
  padding-top: 8px;
  padding-bottom: 8px;
  padding-left: 5px;
  padding-right: 5px;
}

#ybtyhdwwsl .gt_first_grand_summary_row {
  padding-top: 8px;
  padding-bottom: 8px;
  padding-left: 5px;
  padding-right: 5px;
  border-top-style: double;
  border-top-width: 6px;
  border-top-color: #D3D3D3;
}

#ybtyhdwwsl .gt_last_grand_summary_row_top {
  padding-top: 8px;
  padding-bottom: 8px;
  padding-left: 5px;
  padding-right: 5px;
  border-bottom-style: double;
  border-bottom-width: 6px;
  border-bottom-color: #D3D3D3;
}

#ybtyhdwwsl .gt_striped {
  background-color: rgba(128, 128, 128, 0.05);
}

#ybtyhdwwsl .gt_table_body {
  border-top-style: solid;
  border-top-width: 2px;
  border-top-color: #D3D3D3;
  border-bottom-style: solid;
  border-bottom-width: 2px;
  border-bottom-color: #D3D3D3;
}

#ybtyhdwwsl .gt_footnotes {
  color: #333333;
  background-color: #FFFFFF;
  border-bottom-style: none;
  border-bottom-width: 2px;
  border-bottom-color: #D3D3D3;
  border-left-style: none;
  border-left-width: 2px;
  border-left-color: #D3D3D3;
  border-right-style: none;
  border-right-width: 2px;
  border-right-color: #D3D3D3;
}

#ybtyhdwwsl .gt_footnote {
  margin: 0px;
  font-size: 90%;
  padding-top: 4px;
  padding-bottom: 4px;
  padding-left: 5px;
  padding-right: 5px;
}

#ybtyhdwwsl .gt_sourcenotes {
  color: #333333;
  background-color: #FFFFFF;
  border-bottom-style: none;
  border-bottom-width: 2px;
  border-bottom-color: #D3D3D3;
  border-left-style: none;
  border-left-width: 2px;
  border-left-color: #D3D3D3;
  border-right-style: none;
  border-right-width: 2px;
  border-right-color: #D3D3D3;
}

#ybtyhdwwsl .gt_sourcenote {
  font-size: 90%;
  padding-top: 4px;
  padding-bottom: 4px;
  padding-left: 5px;
  padding-right: 5px;
}

#ybtyhdwwsl .gt_left {
  text-align: left;
}

#ybtyhdwwsl .gt_center {
  text-align: center;
}

#ybtyhdwwsl .gt_right {
  text-align: right;
  font-variant-numeric: tabular-nums;
}

#ybtyhdwwsl .gt_font_normal {
  font-weight: normal;
}

#ybtyhdwwsl .gt_font_bold {
  font-weight: bold;
}

#ybtyhdwwsl .gt_font_italic {
  font-style: italic;
}

#ybtyhdwwsl .gt_super {
  font-size: 65%;
}

#ybtyhdwwsl .gt_footnote_marks {
  font-size: 75%;
  vertical-align: 0.4em;
  position: initial;
}

#ybtyhdwwsl .gt_asterisk {
  font-size: 100%;
  vertical-align: 0;
}

#ybtyhdwwsl .gt_indent_1 {
  text-indent: 5px;
}

#ybtyhdwwsl .gt_indent_2 {
  text-indent: 10px;
}

#ybtyhdwwsl .gt_indent_3 {
  text-indent: 15px;
}

#ybtyhdwwsl .gt_indent_4 {
  text-indent: 20px;
}

#ybtyhdwwsl .gt_indent_5 {
  text-indent: 25px;
}

#ybtyhdwwsl .katex-display {
  display: inline-flex !important;
  margin-bottom: 0.75em !important;
}

#ybtyhdwwsl div.Reactable > div.rt-table > div.rt-thead > div.rt-tr.rt-tr-group-header > div.rt-th-group:after {
  height: 0px !important;
}
</style>
  <table class="gt_table" data-quarto-disable-processing="false" data-quarto-bootstrap="false">
  <thead>
    <tr class="gt_col_headings">
      <th class="gt_col_heading gt_columns_bottom_border gt_left" rowspan="1" colspan="1" scope="col" id="label"><span class="gt_from_md"><strong>Characteristic</strong></span></th>
      <th class="gt_col_heading gt_columns_bottom_border gt_center" rowspan="1" colspan="1" scope="col" id="stat_0"><span class="gt_from_md"><strong>N = 4,602</strong></span><span class="gt_footnote_marks" style="white-space:nowrap;font-style:italic;font-weight:normal;line-height:0;"><sup>1</sup></span></th>
    </tr>
  </thead>
  <tbody class="gt_table_body">
    <tr><td headers="label" class="gt_row gt_left" style="font-weight: bold;">Sex</td>
<td headers="stat_0" class="gt_row gt_center"><br></td></tr>
    <tr><td headers="label" class="gt_row gt_left">&nbsp;&nbsp;&nbsp;&nbsp;Female</td>
<td headers="stat_0" class="gt_row gt_center">2,261.0 (49.1%)</td></tr>
    <tr><td headers="label" class="gt_row gt_left">&nbsp;&nbsp;&nbsp;&nbsp;Male</td>
<td headers="stat_0" class="gt_row gt_center">2,341.0 (50.9%)</td></tr>
    <tr><td headers="label" class="gt_row gt_left" style="font-weight: bold;">Age at Diagnosis</td>
<td headers="stat_0" class="gt_row gt_center">61 (46, 73)</td></tr>
    <tr><td headers="label" class="gt_row gt_left" style="font-weight: bold;">Residence Area</td>
<td headers="stat_0" class="gt_row gt_center"><br></td></tr>
    <tr><td headers="label" class="gt_row gt_left">&nbsp;&nbsp;&nbsp;&nbsp;Rural (Parishes)</td>
<td headers="stat_0" class="gt_row gt_center">462.0 (10.0%)</td></tr>
    <tr><td headers="label" class="gt_row gt_left">&nbsp;&nbsp;&nbsp;&nbsp;Urban (Quito)</td>
<td headers="stat_0" class="gt_row gt_center">4,140.0 (90.0%)</td></tr>
    <tr><td headers="label" class="gt_row gt_left" style="font-weight: bold;">Education Level</td>
<td headers="stat_0" class="gt_row gt_center"><br></td></tr>
    <tr><td headers="label" class="gt_row gt_left">&nbsp;&nbsp;&nbsp;&nbsp;None / Illiterate</td>
<td headers="stat_0" class="gt_row gt_center">252.0 (5.5%)</td></tr>
    <tr><td headers="label" class="gt_row gt_left">&nbsp;&nbsp;&nbsp;&nbsp;Primary</td>
<td headers="stat_0" class="gt_row gt_center">1,593.0 (34.6%)</td></tr>
    <tr><td headers="label" class="gt_row gt_left">&nbsp;&nbsp;&nbsp;&nbsp;Secondary</td>
<td headers="stat_0" class="gt_row gt_center">1,426.0 (31.0%)</td></tr>
    <tr><td headers="label" class="gt_row gt_left">&nbsp;&nbsp;&nbsp;&nbsp;Higher (University/Technical)</td>
<td headers="stat_0" class="gt_row gt_center">1,005.0 (21.8%)</td></tr>
    <tr><td headers="label" class="gt_row gt_left">&nbsp;&nbsp;&nbsp;&nbsp;Missing/Unknown</td>
<td headers="stat_0" class="gt_row gt_center">326.0 (7.1%)</td></tr>
    <tr><td headers="label" class="gt_row gt_left" style="font-weight: bold;">Diagnostic Establishment</td>
<td headers="stat_0" class="gt_row gt_center"><br></td></tr>
    <tr><td headers="label" class="gt_row gt_left">&nbsp;&nbsp;&nbsp;&nbsp;Ministry of Public Health (MSP)</td>
<td headers="stat_0" class="gt_row gt_center">753.0 (16.4%)</td></tr>
    <tr><td headers="label" class="gt_row gt_left">&nbsp;&nbsp;&nbsp;&nbsp;Missing</td>
<td headers="stat_0" class="gt_row gt_center">364.0 (7.9%)</td></tr>
    <tr><td headers="label" class="gt_row gt_left">&nbsp;&nbsp;&nbsp;&nbsp;Private Sector</td>
<td headers="stat_0" class="gt_row gt_center">1,050.0 (22.8%)</td></tr>
    <tr><td headers="label" class="gt_row gt_left">&nbsp;&nbsp;&nbsp;&nbsp;Social Security (IESS)</td>
<td headers="stat_0" class="gt_row gt_center">1,464.0 (31.8%)</td></tr>
    <tr><td headers="label" class="gt_row gt_left">&nbsp;&nbsp;&nbsp;&nbsp;Specialized Cancer Center (SOLCA)</td>
<td headers="stat_0" class="gt_row gt_center">971.0 (21.1%)</td></tr>
    <tr><td headers="label" class="gt_row gt_left" style="font-weight: bold;">Histological Subtype</td>
<td headers="stat_0" class="gt_row gt_center"><br></td></tr>
    <tr><td headers="label" class="gt_row gt_left">&nbsp;&nbsp;&nbsp;&nbsp;B-Cell / NOS / Other</td>
<td headers="stat_0" class="gt_row gt_center">1,261.0 (27.4%)</td></tr>
    <tr><td headers="label" class="gt_row gt_left">&nbsp;&nbsp;&nbsp;&nbsp;Burkitt Lymphoma</td>
<td headers="stat_0" class="gt_row gt_center">10.0 (0.2%)</td></tr>
    <tr><td headers="label" class="gt_row gt_left">&nbsp;&nbsp;&nbsp;&nbsp;Diffuse Large B-Cell (DLBCL)</td>
<td headers="stat_0" class="gt_row gt_center">1,989.0 (43.2%)</td></tr>
    <tr><td headers="label" class="gt_row gt_left">&nbsp;&nbsp;&nbsp;&nbsp;Follicular Lymphoma</td>
<td headers="stat_0" class="gt_row gt_center">384.0 (8.3%)</td></tr>
    <tr><td headers="label" class="gt_row gt_left">&nbsp;&nbsp;&nbsp;&nbsp;Hodgkin Lymphoma</td>
<td headers="stat_0" class="gt_row gt_center">495.0 (10.8%)</td></tr>
    <tr><td headers="label" class="gt_row gt_left">&nbsp;&nbsp;&nbsp;&nbsp;Mature T/NK-Cell</td>
<td headers="stat_0" class="gt_row gt_center">463.0 (10.1%)</td></tr>
    <tr><td headers="label" class="gt_row gt_left" style="font-weight: bold;">History of Prior Cancer</td>
<td headers="stat_0" class="gt_row gt_center"><br></td></tr>
    <tr><td headers="label" class="gt_row gt_left">&nbsp;&nbsp;&nbsp;&nbsp;No prior tumor</td>
<td headers="stat_0" class="gt_row gt_center">4,213.0 (91.5%)</td></tr>
    <tr><td headers="label" class="gt_row gt_left">&nbsp;&nbsp;&nbsp;&nbsp;One prior tumor</td>
<td headers="stat_0" class="gt_row gt_center">174.0 (3.8%)</td></tr>
    <tr><td headers="label" class="gt_row gt_left">&nbsp;&nbsp;&nbsp;&nbsp;Two or more prior tumors</td>
<td headers="stat_0" class="gt_row gt_center">215.0 (4.7%)</td></tr>
    <tr><td headers="label" class="gt_row gt_left" style="font-weight: bold;">Primary Tumor Site</td>
<td headers="stat_0" class="gt_row gt_center"><br></td></tr>
    <tr><td headers="label" class="gt_row gt_left">&nbsp;&nbsp;&nbsp;&nbsp;Gastric (Extranodal)</td>
<td headers="stat_0" class="gt_row gt_center">530.0 (11.5%)</td></tr>
    <tr><td headers="label" class="gt_row gt_left">&nbsp;&nbsp;&nbsp;&nbsp;Nodal</td>
<td headers="stat_0" class="gt_row gt_center">3,805.0 (82.7%)</td></tr>
    <tr><td headers="label" class="gt_row gt_left">&nbsp;&nbsp;&nbsp;&nbsp;Other Extranodal</td>
<td headers="stat_0" class="gt_row gt_center">267.0 (5.8%)</td></tr>
  </tbody>
  <tfoot>
    <tr class="gt_footnotes">
      <td class="gt_footnote" colspan="2"><span class="gt_footnote_marks" style="white-space:nowrap;font-style:italic;font-weight:normal;line-height:0;"><sup>1</sup></span> <span class="gt_from_md">n (%); Median (Q1, Q3)</span></td>
    </tr>
  </tfoot>
</table>
</div>